# Optimising Beale function with has a minimum on f(3, 0.5) = 0 for -4.5<x,y<4.5
## These functions should be passed a numpy array of floats to ensure no exceptions are raised!

In [1]:
import numpy as np

In [2]:
def bealeFunction (x, y): # Can do broadcasting if you pass it numpy arrays! All individuals can be evaluated at once
    term1 = (1.5-x+x*y)**2
    term2 = (2.25-x+x*y**2)**2
    term3 = (2.625-x+x*y**3)**2
    return term1+term2+term3

In [3]:
def createRandomIndividuals (numberOfIndividuals, low=-4.5, high=4.5):
    Xs = np.random.uniform (low=low, high = high, size = numberOfIndividuals)
    Ys = np.random.uniform (low=low, high = high, size = numberOfIndividuals)
    return Xs, Ys

In [4]:
def elitismSelection (Xs, Ys, numberToBeChosen):
    scores  = bealeFunction (Xs, Ys)
    indices = np.argsort (scores)
    Xs      = Xs[indices][:numberToBeChosen]
    Ys      = Ys[indices][:numberToBeChosen]
    return Xs, Ys

In [5]:
def rouletteWheel (Xs, Ys, numberToBeChosen, replace=True):
    fitness       = 1/(bealeFunction (Xs, Ys)+0.1) # So we don't divide by 0

    totalFitness  = np.sum(fitness)
    probabilities = fitness/totalFitness

    chosenIndices = np.random.choice(range(len(fitness)), size=numberToBeChosen, replace=replace, p=probabilities)
    return Xs[chosenIndices], Ys[chosenIndices]

In [6]:
def tournament (Xs, Ys, numberToBeChosen, individualsPerTournament=4, replace=False):
    fitness = 1/(bealeFunction (Xs, Ys)+0.1) # So we don't divide by 0
    chosenIndices=[]

    for i in range (numberToBeChosen):
        competingIndices = np.random.choice(range(len(fitness)), replace=replace, size=individualsPerTournament)
        chosenIndices.append(max(competingIndices, key = lambda x: fitness[x]))

    return Xs[chosenIndices], Ys[chosenIndices]

In [7]:
def createPairs (numberOfSurvivors, numberOfPairs, replace=True): # It returns a list of indices that can be splitted every 2 for forming pairs
    pairs = []
    chosenIndices = np.random.choice (range(numberOfSurvivors), size=(numberOfPairs*2), replace = replace)
    return chosenIndices

In [8]:
def SBX (parameters1, parameters2, eta=100): # Simulated Binary Crossover. The bigger is eta, the more close the children are to parents
    r             = np.random.random(size=len(parameters1))
    gamma         = np.empty(len(parameters1))
    gamma[r<=0.5] = (2*r[r<=0.5])**(1/(eta+1))
    gamma[r>0.5 ] = (1/(2*(1-r[r>0.5])))**(1/(eta+1))
    child1        = 0.5 * ((1+gamma)*parameters1 + (1-gamma)*parameters2)
    child2        = 0.5 * ((1-gamma)*parameters1 + (1+gamma)*parameters2)
    return child1, child2

In [9]:
def SPBX (parameters1, parameters2): # Single Point Binary Crossover.
    crossoverPoint = np.random.randint(1, len(parameters1)) # We want at least length 1 at either side of the crossover
    child1         = np.append(parameters1[:crossoverPoint], parameters2[crossoverPoint:])
    child2         = np.append(parameters2[:crossoverPoint], parameters1[crossoverPoint:])
    return child1, child2

In [10]:
def applyGaussianMutation (parameters, mutation_rate=0.05, mu=0, sigma=1): # Pass it a numpy array of floats! (mu and sigma can be arrays too)
    mutationOrNot              = (np.random.random (size=len(parameters))) < mutation_rate
    mutationValues             = np.random.normal(loc=mu, scale = sigma, size = len(parameters))
    parameters[mutationOrNot] += mutationValues[mutationOrNot]
    return parameters

In [11]:
def applyUniformMutation (parameters, mutation_rate=0.05, low=0, high = 1): # Pass it a numpy array of floats! (low and high can be arrays too)
    mutationOrNot              = (np.random.random (size=len(parameters))) < mutation_rate
    mutationValues             = np.random.uniform (low = low, high = high, size = len(parameters))
    parameters[mutationOrNot] += mutationValues[mutationOrNot]
    return parameters

In [12]:
def applyUniformWRTBestIndividualMutation(parameters, bestIndividualParameters, mutation_rate=0.05): # Pass it a numpy array of floats!
    # In this case low will always be 0 and high will always be 1. We are measuring how much the parameters will be moved towards those of the best individual
    mutationOrNot              = (np.random.random (size=len(parameters))) < mutation_rate
    mutationPercentage         = np.random.uniform (size = len(parameters))
    difference                 = parameters - bestIndividualParameters
    parameters[mutationOrNot] -= difference[mutationOrNot] * mutationPercentage[mutationOrNot]
    return parameters

In [15]:
# The following variables mean that each generation has 100 individuals. 20 fittest individuals are selected to have children
# In each generation, 80 children will be created from crossover plus mutation, 10 parents will remain and 10 new individuals will be created randonmly
# 20 trials will be made to see if the results are consistent.

numberOfTrials = 20

numberOfGenerations = 500
indivPerGen         = 100
fittestNumber       = 20

childrenFromParents = 80
parentsThatRemain   = 10
numberNewIndividuals      = 10

mu = 0
sigma = 0.1

for _ in range (numberOfTrials):
    
    primitiveGeneration = np.stack(createRandomIndividuals (numberOfIndividuals=indivPerGen, low=-4.5, high=4.5))
    # Shape (2, indivPerGen). Contains Xs and Ys

    pastGeneration   = np.copy(primitiveGeneration)
    
    for i in range (numberOfGenerations):
        
        # Choose one selection type:
        
        #survivors    = np.stack(rouletteWheel (pastGeneration[0], pastGeneration[1], numberToBeChosen=fittestNumber, replace=True))
        #survivors    = np.stack(tournament (pastGeneration[0], pastGeneration[1], numberToBeChosen=fittestNumber, individualsPerTournament=4, replace=False))
        survivors    = np.stack(elitismSelection (pastGeneration[0], pastGeneration[1], numberToBeChosen=fittestNumber))
        # Shape (2, fittestNumber). Contains Xs and Ys

        pairs        = createPairs (numberOfSurvivors=fittestNumber, numberOfPairs = childrenFromParents//2, replace=True)
        # Shape (childrenFromParents,). Contains indices for survivors Array ready to be splitted every two.

        newChildren  = []
        
        for i in range (childrenFromParents//2):
            
            parents  = survivors[:,pairs[2*i:2*(i+1)]]
            # Shape (2, numberOfParameters =2)
            
            child1, child2 = SBX (parents[0], parents[1]) 
            
            # Choose one mutation type:
            
            #child1, child2 = applyUniformMutation (child1, low=-2.5*sigma, high = 2.5*sigma), applyUniformMutation (child2, low =-2.5*sigma, high = 2.5*sigma)
            child1, child2 = applyGaussianMutation (child1, mu=mu, sigma=sigma),              applyGaussianMutation (child2, mu=mu, sigma=sigma)
            #child1, child2 = applyUniformWRTBestIndividualMutation(child1,survivors[:,1]),    applyUniformWRTBestIndividualMutation(child2,survivors[:,1])
            
            newChildren.append(child1)
            newChildren.append(child2)
            
        newChildren    = np.swapaxes(newChildren, axis1=1, axis2=0)
        # Shape (2, ChildrenFromParents). Contains Xs and Ys

        newIndividuals = np.stack(createRandomIndividuals (numberNewIndividuals, low=-4.5, high=4.5))
        # Shape (2, numberNewIndividuals)

        newGeneration  = np.concatenate ((survivors[:,:parentsThatRemain],newChildren, newIndividuals), axis = 1)
        # Shape (2, indivPerGen)

        pastGeneration = np.copy(newGeneration)

    bestIndividual     = np.stack(elitismSelection (newGeneration[0], newGeneration[1], numberToBeChosen=1))
    print ("The best individual is X:%.4f, Y:%.4f with a function value of %.4f" % (bestIndividual[0], bestIndividual[1],bealeFunction(bestIndividual[0], bestIndividual[1])))

The best individual is X:3.0101, Y:0.5024 with a function value of 0.0000
The best individual is X:3.0142, Y:0.5040 with a function value of 0.0000
The best individual is X:3.0067, Y:0.5015 with a function value of 0.0000
The best individual is X:3.0019, Y:0.5009 with a function value of 0.0000
The best individual is X:3.1048, Y:0.5237 with a function value of 0.0016
The best individual is X:3.0053, Y:0.5012 with a function value of 0.0000
The best individual is X:3.0069, Y:0.5006 with a function value of 0.0000
The best individual is X:2.9998, Y:0.4995 with a function value of 0.0000
The best individual is X:3.0401, Y:0.5120 with a function value of 0.0004
The best individual is X:3.0489, Y:0.5143 with a function value of 0.0005
The best individual is X:3.1153, Y:0.5293 with a function value of 0.0020
The best individual is X:3.1914, Y:0.5424 with a function value of 0.0048
The best individual is X:3.0947, Y:0.5195 with a function value of 0.0015
The best individual is X:2.9677, Y:0.4